Questão 1

1 - Realtime Architecture

a) Para transformarmos a arquitetura em tempo real, precisaríamos de outra peça que também seria Consumer do Kafka. No caso, poderia ser um Kakfa Stream que toma alguma ação com base nos streaming de entrada.

b) Nessa arquitetura, é possível ser mais rápido e efetivo quando um dado trigger é acionado, atuando em tempo real sobre uma falha ou inconsistência de um processo, coisa que num processo batch demoraria mais para ser descoberto ou executado. Além disso, em certos casos, não existe a necessidade de ter um banco de dados associado.

##Problema: Eficácia da Mídia Digital

O que você precisa responder:

(Preparar o banco de dados/tabelas analíticas para realizar consultas)

1) Qual foi a campanha mais cara?
2) Qual foi a campanha mais lucrativa?
3) Qual o anúncio é o mais eficaz em termos de cliques?
4) Qual anúncio é o mais eficas em termos de geração de leads?

O que você precisa fazer:

1) Ingerir os dados em um banco de dados
2) Fornecer uma única tabela pronta para ser usada para responder às perguntas acima
3) Fornecer as querys SQL no README (que são as respostas pra cada questão comercial acima)
4) Explicar como reproduzir sua solução


Carregando as bases

In [1]:
import pandas as pd
import json
import re

## REGEX para o pageview ##

ip = r"\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}"
datetime = r"\d\d\d\d\-\d+\-\d+\s\d\d\:\d\d\:\d\d"
url = r"http.+"
campaign_id = r"campaign_id=[0-9]+"
creative_id = r"ad_creative_id=[0-9]+"
deviceId = r"device_id: \w+"
notDeviceId = r"[^device_id: ]+"
referer = r"http[^ ]+$"

In [2]:
with open('google_ads_media_costs.jsonl', 'r') as json_file:
    json_list = list(json_file)

with open('google_ads_media_costs.jsonl') as f:
    data = [json.loads(line) for line in f]

googleAdsMediaCosts = pd.read_json(path_or_buf="google_ads_media_costs.jsonl", lines=True)
googleAdsMediaCosts.columns = ["DATE", "CAMPAIGN_ID", "CAMPAIGN_NAME", "AD_CREATIVE_ID", "AD_CREATIVE_NAME", "CLICKS", "IMPRESSIONS", "COST"]
googleAdsMediaCosts["SRC"] = "google"

facebookAdsMediaCosts = pd.read_json(path_or_buf="facebook_ads_media_costs.jsonl", lines=True)
facebookAdsMediaCosts.columns = ["DATE", "CAMPAIGN_ID", "CAMPAIGN_NAME", "CLICKS", "IMPRESSIONS", "COST"]
facebookAdsMediaCosts["SRC"] = "facebook"

customer_leads_funnel = pd.read_csv("customer_leads_funnel.csv", names = ["ID_DEVICE", "ID_LEAD", "REGISTERED_AT", "CREDIT_DECISION", "CREDIT_DECISION_AT", "SIGNED_AT", "REVENUE"])

pageview = []

with open("pageview.txt", 'r') as txt_file:
  for i in txt_file:
    i = i.replace("\n", "")
    pageview.append(i.split("|"))

line = []
matrix = []

for i in pageview:

  line.append(re.findall(ip, i[0])[0])
  line.append(re.findall(datetime, i[0])[0])
  line.append(re.findall(url, i[0])[0])
  try:
    line.append(re.findall(r"[0-9]+", re.findall(campaign_id, line[2])[0])[0])
  except:
    line.append("")
  try:
    line.append(re.findall(r"[0-9]+", re.findall(creative_id, line[2])[0])[0])
  except:
    line.append("")
  line.append(re.findall(notDeviceId, i[1])[0])
  line.append(re.findall(referer, i[2])[0])

  matrix.append(line)

  line = []

#['169.112.4.32', '2018-10-23 08:19:36', 'http://www.creditas.com.br/emprestimo?ad_creative_id=20001&campaign_id=1006
cols = ["IP", "DATE", "URL", "ID_CAMPAIGN", "ID_CREATIVE", "ID_DEVICE", "REFERER"]
pageviewDataFrame = pd.DataFrame(matrix, columns = cols)

Carregando bases do Sqlite

In [3]:
#Códigos para o SQL

dropADSMedia = "drop table if exists ADS_MEDIA_COSTS;"
truncateADSMedia = "truncate table ADS_MEDIA_COSTS;"

createADSMedia = """

CREATE TABLE ADS_MEDIA_COSTS (
  
    ID_ADS_MEDIA_COSTS INTEGER PRIMARY KEY AUTOINCREMENT,
    DATE datetime, 
    CAMPAIGN_ID INT,
    CAMPAIGN_NAME NVARCHAR(255),
    AD_CREATIVE_ID INT, 
    AD_CREATIVE_NAME NVARCHAR(255),
    CLICKS INT, 
    IMPRESSIONS INT,
    COST REAL,
    SRC nvarchar(20)
    
  );"""

createPageview = """

CREATE TABLE PAGEVIEW (
    
    ID_PAGEVIEW INTEGER PRIMARY KEY AUTOINCREMENT,
    IP NVARCHAR(50),
    DATE datetime,
    URL nvarchar(255),
    ID_CAMPAIGN INT,
    ID_CREATIVE INT,
    ID_DEVICE nvarchar(50),
    REFERER nvarchar(255)
    
 );"""

createCustomerLeadsFunnel = """

CREATE TABLE CUSTOMER_LEADS_FUNNEL (
  
  ID_CUSTOMER_LEADS_FUNNEL INTEGER PRIMARY KEY AUTOINCREMENT,
  ID_DEVICE INT,
  ID_LEAD INT,
  REGISTERED_AT DATETIME,
  CREDIT_DECISION NVARCHAR(5),
  CREDIT_DECISION_AT DATETIME,
  SIGNED_AT DATETIME,
  REVENUE REAL

);"""


In [4]:
import sqlite3

#Cria tabelas

con = sqlite3.connect("db.sqlite")

con.execute(createADSMedia)
con.execute(createPageview)
con.execute(createCustomerLeadsFunnel)

con.close()

In [5]:
#Carrega os dados nas tabelas

con = sqlite3.connect("db.sqlite")

googleAdsMediaCosts.to_sql('ADS_MEDIA_COSTS', con=con, if_exists='append', index= False)
facebookAdsMediaCosts.to_sql('ADS_MEDIA_COSTS', con=con, if_exists='append', index= False)
pageviewDataFrame.to_sql('PAGEVIEW', con=con, if_exists='append', index= False)
customer_leads_funnel.to_sql('CUSTOMER_LEADS_FUNNEL', con=con, if_exists='append', index= False)

con.close()

In [6]:
#Checa se inseriu os dados e se houve perdas no caminho

con = sqlite3.connect("db.sqlite")
cur = con.cursor()

for row in cur.execute("SELECT count(*) FROM ADS_MEDIA_COSTS"):
    print(row[0] == (len(googleAdsMediaCosts) + len(facebookAdsMediaCosts)))
for row in cur.execute("SELECT count(*) FROM PAGEVIEW"):
    print(row[0] == len(pageviewDataFrame))
for row in cur.execute("SELECT count(*) FROM CUSTOMER_LEADS_FUNNEL"):
    print(row[0] == len(customer_leads_funnel))

con.close()

True
True
True


Analisando e respondendo as perguntas

In [7]:
#Campanha mais cara

con = sqlite3.connect("db.sqlite")
cur = con.cursor()

for row in cur.execute("SELECT SRC, CAMPAIGN_ID, SUM(COST) FROM ADS_MEDIA_COSTS GROUP BY CAMPAIGN_ID, SRC ORDER BY SUM(COST) DESC LIMIT 1"):
    print(row)

con.close()

('google', 1004, 15626.58)


In [8]:
#Campanha mais lucrativa

con = sqlite3.connect("db.sqlite")
cur = con.cursor()

#maior lucro
#for row in cur.execute("SELECT ID_CAMPAIGN, SUM(REVENUE) AS RECEITA FROM PAGEVIEW A INNER JOIN CUSTOMER_LEADS_FUNNEL B ON a.ID_DEVICE = b.ID_DEVICE WHERE ID_CAMPAIGN != '' group by ID_CAMPAIGN order by SUM(REVENUE) DESC"):
#    print(row)

for row in cur.execute("SELECT ID_CAMPAIGN, (RECEITA-CUSTO) AS LUCRO FROM (SELECT ID_CAMPAIGN, SUM(REVENUE) AS RECEITA FROM PAGEVIEW A INNER JOIN CUSTOMER_LEADS_FUNNEL B ON a.ID_DEVICE = b.ID_DEVICE WHERE ID_CAMPAIGN != '' group by ID_CAMPAIGN order by SUM(REVENUE) DESC) A inner join (SELECT CAMPAIGN_ID, SUM(COST) AS CUSTO FROM ADS_MEDIA_COSTS GROUP BY CAMPAIGN_ID, SRC ORDER BY SUM(COST) DESC) B on A.ID_CAMPAIGN = B.CAMPAIGN_ID LIMIT 1"):
    print(row)

con.close()

(1009, 10060.610000000002)


In [9]:
#Maior efetividade em cliques
## OBS: Apenas os anúncios do google tem informações caracterizando o AD.

con = sqlite3.connect("db.sqlite")
cur = con.cursor()

for row in cur.execute("SELECT AD_CREATIVE_ID, SUM(CLICKS) FROM ADS_MEDIA_COSTS where SRC = 'google' group by AD_CREATIVE_ID order by SUM(CLICKS) desc LIMIT 1"):
    print(row)

# Mas podemos agrupar cliques por campanha também:
#for row in cur.execute("SELECT CAMPAIGN_ID, SUM(CLICKS) FROM ADS_MEDIA_COSTS where SRC = 'facebook' group by CAMPAIGN_ID order by SUM(CLICKS) desc LIMIT 1"):
#    print(row)

con.close()

(20007, 125055489)


In [126]:
#Maior efetividade em gerar leads

con = sqlite3.connect("db.sqlite")
cur = con.cursor()

for row in cur.execute("SELECT COUNT(ID_CREATIVE), ID_CREATIVE FROM PAGEVIEW WHERE ID_CREATIVE != '' AND ID_DEVICE IN (SELECT DISTINCT ID_DEVICE FROM CUSTOMER_LEADS_FUNNEL) GROUP BY ID_CREATIVE ORDER BY COUNT(ID_CREATIVE) DESC LIMIT 1"):
    print(row)

con.close()

(445, 20003)


In [39]:
#Gerando tabelão final

#Campanha mais lucrativa

con = sqlite3.connect("db.sqlite")

con.execute(
"""

create table IF NOT EXISTS AD_CUSTOMER AS

select ID_ADS_MEDIA_COSTS, ID_PAGEVIEW, ID_CUSTOMER_LEADS_FUNNEL, A.DATE, CAMPAIGN_ID, CAMPAIGN_NAME, AD_CREATIVE_ID, AD_CREATIVE_NAME, CLICKS, IMPRESSIONS, COST, SRC,
IP, B.DATE, URL, ID_CAMPAIGN, ID_CREATIVE, B.ID_DEVICE, REFERER, ID_LEAD, REGISTERED_AT, CREDIT_DECISION, CREDIT_DECISION_AT, SIGNED_AT, REVENUE
from ADS_MEDIA_COSTS A INNER JOIN PAGEVIEW B ON A.CAMPAIGN_ID = B.ID_CAMPAIGN INNER JOIN CUSTOMER_LEADS_FUNNEL C ON B.ID_DEVICE = C.ID_DEVICE

"""
)


con.close()

In [42]:
##Campanha mais cara NO TABELÃO

con = sqlite3.connect("db.sqlite")
cur = con.cursor()

for row in cur.execute("SELECT SUM(COST), CAMPAIGN_ID FROM (SELECT DISTINCT ID_ADS_MEDIA_COSTS, CAMPAIGN_ID, COST FROM AD_CUSTOMER) GROUP BY CAMPAIGN_ID ORDER BY SUM(COST) DESC LIMIT 1"):
    print(row)

con.close()

(15626.58, 1004)


In [48]:
##Maior lucro NO TABELÃO

con = sqlite3.connect("db.sqlite")
cur = con.cursor()

for row in cur.execute("SELECT A.CAMPAIGN_ID, (RECEITA-CUSTO) AS LUCRO FROM (SELECT SUM(COST) AS CUSTO, CAMPAIGN_ID FROM (SELECT DISTINCT ID_ADS_MEDIA_COSTS, CAMPAIGN_ID, COST FROM AD_CUSTOMER) GROUP BY CAMPAIGN_ID) A inner join (SELECT SUM(REVENUE) AS RECEITA, CAMPAIGN_ID FROM (SELECT DISTINCT ID_CUSTOMER_LEADS_FUNNEL, CAMPAIGN_ID, REVENUE FROM AD_CUSTOMER) GROUP BY CAMPAIGN_ID) B ON A.CAMPAIGN_ID = B.CAMPAIGN_ID ORDER BY LUCRO DESC LIMIT 1"):
    print(row)

con.close()

(1009, 10060.609999999999)


In [51]:
##Maior efetividade em cliques NO TABELAO

con = sqlite3.connect("db.sqlite")
cur = con.cursor()

for row in cur.execute("SELECT SUM(CLICKS), AD_CREATIVE_ID FROM (SELECT DISTINCT ID_ADS_MEDIA_COSTS, AD_CREATIVE_ID, CLICKS FROM AD_CUSTOMER WHERE AD_CREATIVE_ID IS NOT NULL) GROUP BY AD_CREATIVE_ID ORDER BY SUM(CLICKS) DESC LIMIT 1"):
    print(row)

con.close()

(125055489, 20007)


In [125]:
##Maior efetividade em leads NO TABELAO

con = sqlite3.connect("db.sqlite")
cur = con.cursor()

for row in cur.execute("SELECT ID_CREATIVE, COUNT(ID_CREATIVE) FROM (SELECT ID_CREATIVE FROM AD_CUSTOMER WHERE ID_CREATIVE != '' AND ID_DEVICE IN (SELECT ID_DEVICE FROM AD_CUSTOMER GROUP BY ID_CUSTOMER_LEADS_FUNNEL, ID_PAGEVIEW, ID_DEVICE) GROUP BY ID_DEVICE) GROUP BY ID_CREATIVE ORDER BY COUNT(ID_CREATIVE) DESC LIMIT 1"):
    print(row)

con.close()

(20003, 445)


In [127]:
### respostas ###

#1 - 1004
#2 - 1009
#3 - 20007
#4 - 20003

### querys ###

#1 - SELECT SUM(COST), CAMPAIGN_ID FROM (SELECT DISTINCT ID_ADS_MEDIA_COSTS, CAMPAIGN_ID, COST FROM AD_CUSTOMER) GROUP BY CAMPAIGN_ID ORDER BY SUM(COST) DESC LIMIT 1
#2 - SELECT A.CAMPAIGN_ID, (RECEITA-CUSTO) AS LUCRO FROM (SELECT SUM(COST) AS CUSTO, CAMPAIGN_ID FROM (SELECT DISTINCT ID_ADS_MEDIA_COSTS, CAMPAIGN_ID, COST FROM AD_CUSTOMER) GROUP BY CAMPAIGN_ID) A inner join (SELECT SUM(REVENUE) AS RECEITA, CAMPAIGN_ID FROM (SELECT DISTINCT ID_CUSTOMER_LEADS_FUNNEL, CAMPAIGN_ID, REVENUE FROM AD_CUSTOMER) GROUP BY CAMPAIGN_ID) B ON A.CAMPAIGN_ID = B.CAMPAIGN_ID ORDER BY LUCRO DESC LIMIT 1
#3 - SELECT SUM(CLICKS), AD_CREATIVE_ID FROM (SELECT DISTINCT ID_ADS_MEDIA_COSTS, AD_CREATIVE_ID, CLICKS FROM AD_CUSTOMER WHERE AD_CREATIVE_ID IS NOT NULL) GROUP BY AD_CREATIVE_ID ORDER BY SUM(CLICKS) DESC LIMIT 1
#4 - SELECT ID_CREATIVE, COUNT(ID_CREATIVE) FROM (SELECT ID_CREATIVE FROM AD_CUSTOMER WHERE ID_CREATIVE != '' AND ID_DEVICE IN (SELECT ID_DEVICE FROM AD_CUSTOMER GROUP BY ID_CUSTOMER_LEADS_FUNNEL, ID_PAGEVIEW, ID_DEVICE) GROUP BY ID_DEVICE) GROUP BY ID_CREATIVE ORDER BY COUNT(ID_CREATIVE) DESC LIMIT 1

Questão 2

Athena é uma engine para fazer consultas em dados armazenados em S3. Já o Redshift Spectrum é utilizado junto as ferramentas do AWS Redshift. Além disso, sobre o Redshift é possível ter controle sobre a alocação dee recursos, enquando que no caso do Athena é alocado automaticamente. E, ainda, de forma macro, o Redshift Spectrum consegue lidar melhor com grandes querys e quantidades de dados.


Questão 3

A maior diferença é o fato do ETL carregar os dados de um sistema origem, transformá-los no meio do caminho e levar ao sistema destino. Já o ELT carrega os dados diretamente no destino e os transforma lá.

Dessa forma, quando existe um volume de dados muito grande, é mais vantajoso utilizar o padrão ELT.

##Parte 2

Q4 - ⦁	Consolida a quantidade de clientes ativos e o valor transacionado por data de transação

Q5 ⦁	Integrantes da etapa de transformação 


Q6 ⦁	ETL 

Q7: Hadoop e Cassandra.

Q8 ⦁	Google BigQuery, Apache Spark, Amazon Redshift 


Q9 ⦁	Avro

Q10 ⦁	Apache Kafka 
